<a href="https://colab.research.google.com/github/SumathiGit/Python-World/blob/main/RNN_Text_Generation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.cuda as cuda
import torch.optim as optim

import numpy as np

In [ ]:
import os
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        
        # This is very english language specific
        # We will ingest only these characters:
        self.whitelist = [chr(i) for i in range(32, 127)]
        
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r',  encoding="utf8") as f:
            tokens = 0
            for line in f:
                line = ''.join([c for c in line if c in self.whitelist])
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r',  encoding="utf8") as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                line = ''.join([c for c in line if c in self.whitelist])
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids

In [ ]:
!ls data

train.txt  valid.txt


In [ ]:
corpus = Corpus('./data')

In [ ]:
print(corpus.dictionary.idx2word[10])                                          #We can able to view a particular vocabulary by their index value 
print(corpus.dictionary.word2idx['That'])                                      #Here we can get the index value of a particular word itself

That
10


In [ ]:
print(corpus.train.size())
print(corpus.valid.size())

torch.Size([1039900])
torch.Size([63420])


In [ ]:
id = corpus.train[112]                                                                                        #Here the trained dataset with a index of 112 contains the word else
corpus.dictionary.idx2word[id]

'else'

In [ ]:
vocab_size = len(corpus.dictionary)
print(vocab_size)                                                        #There are 74010 vocabulary in this corpus

74010


In [ ]:
import torch.nn as nn
from torch.autograd import Variable

class RNNModel(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, dropout=0.5):                                       #We are applying dropout for the model do not get overfitting
        
        super(RNNModel, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embed_size)                                                                #First hidden layer in RNN is an Embeeded layer which has three arguments (input dimensions)
        self.drop1 = nn.Dropout(dropout)
        self.drop2 = nn.Dropout(dropout)
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, vocab_size)

        self.init_weights()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop1(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop2(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return Variable(weight.new(self.num_layers, batch_size, self.hidden_size).zero_())

In [ ]:
def batchify(data, batch_size):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // batch_size
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * batch_size)
    # Evenly divide the data across the bsz batches.
    data = data.view(batch_size, -1).t().contiguous()
    if cuda.is_available():
        data = data.cuda()
    return data

In [ ]:
dummy_data = "Once upon a time there was a good king and a queen"
dummy_data_idx = [corpus.dictionary.word2idx[w] for w in dummy_data.split()]
dummy_tensor = torch.LongTensor(dummy_data_idx) 
op = batchify(dummy_tensor, 2)                             #2 indicates the batch size >> the output contains two batches as you can see
for row in op:
    print("%10s %10s" %  (corpus.dictionary.idx2word[row[0]], corpus.dictionary.idx2word[row[1]]))

      Once          a
      upon       good
         a       king
      time        and
     there          a
       was      queen


In [ ]:
bs_train = 20       # batch size for training set
bs_valid = 10       # batch size for validation set
bptt_size = 35      # number of times to unroll the graph for back propagation through time
clip = 0.25         # gradient clipping to check exploding gradient

embed_size = 200    # size of the embedding vector
hidden_size = 200   # size of the hidden state in the RNN 
num_layers = 2      # number of RNN layres to use
dropout_pct = 0.5   # %age of neurons to drop out for regularization

In [ ]:
train_data = batchify(corpus.train, bs_train)
val_data = batchify(corpus.valid, bs_valid)

In [ ]:
train_data.shape

torch.Size([51995, 20])

In [ ]:
model = RNNModel(vocab_size, embed_size, hidden_size, num_layers, dropout_pct)

if cuda.is_available():
    model.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
def get_batch(source, i, evaluation=False):
    seq_len = min(bptt_size, len(source) - 1 - i)
    data = Variable(source[i:i+seq_len], volatile=evaluation)
    target = Variable(source[i+1:i+1+seq_len].view(-1))
    if cuda.is_available():
        data = data.cuda()
        target = target.cuda()
    return data, target

In [ ]:
data, target = get_batch(train_data, 1)

In [ ]:
data.shape

torch.Size([35, 20])

In [ ]:
target.shape

torch.Size([700])

In [ ]:
def train(data_source, lr):
    # Turn on training mode which enables dropout.
    
    model.train()
    total_loss = 0
    hidden = model.init_hidden(bs_train)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for batch, i in enumerate(range(0, data_source.size(0) - 1, bptt_size)):                                                 #Here we are using enumurate to get batch ids instead of fetching batches
        
        data, targets = get_batch(data_source, i)                                                                            #Passing the index as well as the source to get the target as well as the output data

        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = Variable(hidden.data)
        
        if cuda.is_available():
            hidden = hidden.cuda()
        
        # model.zero_grad()
        optimizer.zero_grad()
        
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        
        optimizer.step()
        total_loss += len(data) * loss.data
        
    return total_loss / len(data_source)

In [ ]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout. 
    model.eval()                                 #Put the model in the evaluation mode for validation set here there are no optimizers 
    total_loss = 0
    hidden = model.init_hidden(bs_valid)
    
    for i in range(0, data_source.size(0) - 1, bptt_size):
        data, targets = get_batch(data_source, i, evaluation=True)
        
        if cuda.is_available():
            hidden = hidden.cuda()
            
        output, hidden = model(data, hidden)
        output_flat = output.view(-1, vocab_size)
        
        total_loss += len(data) * criterion(output_flat, targets).data
        hidden = Variable(hidden.data)
        
    return total_loss / len(data_source)


In [ ]:
# Loop over epochs.
best_val_loss = None

In [ ]:
def run(epochs, lr):
    global best_val_loss
    
    for epoch in range(0, epochs):
        train_loss = train(train_data, lr)
        val_loss = evaluate(val_data)
        print("Train Loss: ", train_loss, "Valid Loss: ", val_loss)

        if not best_val_loss or val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "./4.model.pth")                                                #we are saving the best val loss in the file system so we can access the model using the file


In [ ]:
run(5, 0.001)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Train Loss:  tensor(6.1155, device='cuda:0') Valid Loss:  tensor(6.7295, device='cuda:0')
Train Loss:  tensor(6.0440, device='cuda:0') Valid Loss:  tensor(6.7363, device='cuda:0')
Train Loss:  tensor(5.9873, device='cuda:0') Valid Loss:  tensor(6.7455, device='cuda:0')
Train Loss:  tensor(5.9433, device='cuda:0') Valid Loss:  tensor(6.7667, device='cuda:0')
Train Loss:  tensor(5.9047, device='cuda:0') Valid Loss:  tensor(6.7848, device='cuda:0')


In [ ]:
run(5, 0.001)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Train Loss:  tensor(5.8783, device='cuda:0') Valid Loss:  tensor(6.7926, device='cuda:0')
Train Loss:  tensor(5.8554, device='cuda:0') Valid Loss:  tensor(6.8223, device='cuda:0')
Train Loss:  tensor(5.8385, device='cuda:0') Valid Loss:  tensor(6.8556, device='cuda:0')
Train Loss:  tensor(5.8248, device='cuda:0') Valid Loss:  tensor(6.8836, device='cuda:0')
Train Loss:  tensor(5.8185, device='cuda:0') Valid Loss:  tensor(6.8931, device='cuda:0')


In [ ]:
num_words = 200
temperature = 1                                                            #Gramatically correct sentences >> temparature

In [ ]:
model = RNNModel(vocab_size, embed_size, hidden_size, num_layers, dropout_pct)
model.load_state_dict(torch.load("./4.model.pth"))                                                           #Passing the trained file system for evalation

if cuda.is_available():
    model.cuda()
    
model.eval()

RNNModel(
  (encoder): Embedding(74010, 200)
  (drop1): Dropout(p=0.5, inplace=False)
  (drop2): Dropout(p=0.5, inplace=False)
  (rnn): GRU(200, 200, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=200, out_features=74010, bias=True)
)

In [ ]:
# https://nlp.stanford.edu/blog/maximum-likelihood-decoding-with-rnns-the-good-the-bad-and-the-ugly/
# Which sample is better? It depends on your personal taste. The high temperature 
# sample displays greater linguistic variety, but the low temperature sample is 
# more grammatically correct. Such is the world of temperature sampling - lowering 
# the temperature allows you to focus on higher probability output sequences and 
# smooth over deficiencies of the model.

# If we set a high temperature, we can get more entropic (*noisier*) probabilities
# Often we want to sample with low temperatures to produce sharp probabilities
temperature = 0.8

In [ ]:
hidden = model.init_hidden(1)                                                #Initially it was zero in hidden we are initializing with 1
idx = corpus.dictionary.word2idx['Beautiful']                                        #The word we want to start with 
input = Variable(torch.LongTensor([[idx]]).long(), volatile=True)

if cuda.is_available():
    input.data = input.data.cuda()

print(corpus.dictionary.idx2word[idx], '', end='')

for i in range(num_words):                                                     #The num of words we are setting is about 200 i this case 
    output, hidden = model(input, hidden)
    word_weights = output.squeeze().data.div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    input.data.fill_(word_idx)
    word = corpus.dictionary.idx2word[word_idx]

    if word == '<eos>':
        print('')
    else:
        print(word + ' ', end='')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Beautiful 
QUEEN. Your FRANCE. 
Indeed, the better is to him. 
PISTOL. I shall not be it a matter? 
EMILIA. I would not to you be to your bags; 
And for the true my kind of parting before 
The very mind; I have lost mine day, 
And now I could be in a dark 
That tonight. 
TRINCULO. With this ere the poor officer, 
Hath thou no gods not desire to kill the sweet. 
If you see your difference to see me. 
BEROWNE. Why was a gentleman of such a man; 
Which that have power upon my Highness Yet 
The noble sun that saw their stir to victory. 
My heart is with him. 
CELIA. It is the simple Let's to all well; and I have my name? 
ISABELLA. I'll bear our little common eye of roughly to 
strength to choke it. I have call'd the herd of thee. 
ESCALUS. Here is the place of your Grace but whom he will have to 
his love. 
POMPEY. Then I will do sure stay upon your 5. 
TITUS. If he I should save this 